In [27]:
# https://www.analyticsvidhya.com/blog/2021/05/tuning-the-hyperparameters-and-layers-of-neural-network-deep-learning/
# Bayesian Optimization(BO)
# : Grid Search 처럼 모든 경우를 다 계산하는 것이 아니라, 몇개만 계산해서 objective function 의 최대 or 최소가 될 수 있는 hyperparameter 를 찾는 최적화기법

In [28]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from sklearn.preprocessing import StandardScaler

from tensorflow import keras

from numpy.random import seed # 예측 결과 일정하게 하기 위함
seed(1) # 예측 결과 일정하게 하기 위함
import tensorflow as tf  # 예측 결과 일정하게 하기 위함
tf.random.set_seed(2) # 예측 결과 일정하게 하기 위함


In [29]:
# This code makes accuracy the scorer metric.

score_acc = make_scorer(accuracy_score)

In [30]:
# Loaddata set

data = pd.read_csv("bladder_cancer.csv")
data.head(3)

data = data.dropna()

X = data.drop(columns=['Label'], axis=1)
y = data['Label']

In [31]:
# train_set with cross-validation : test_set = 80 : 20 

train_feature, test_feature, train_label, test_label = train_test_split(X, y, test_size = 0.2,random_state=0)

In [32]:
# feature normalization, label 은 normalization 진행하지 않았음
scaler = StandardScaler() # scaler 객체 생성
scaler.fit(train_feature) # train_feature 의 mean 과 standard deviation 값을 추출
train_feature_scaled = scaler.transform(train_feature) # train_feature 의 정규화 진행
test_feature_scaled = scaler.transform(test_feature) # test_feature 의 정규화 진행.
# test_feature 는 mean 과 standard deviation 값을 추출하는 과정 하면 안됨. 
# 학습할 때와 동일한 기반 설정으로 동일하게 테스트 데이터를 변환되야 함. 

In [33]:
# pandas numpy 로 변환
train_label = np.array(train_label)
test_label = np.array(test_label)

In [34]:
# Create function
def nn_cl_bo2(neurons_1st_hidden,neurons_other_hidden_1,neurons_other_hidden_2, learning_rate, batch_size, epochs,
              layers1, layers2):
   
    optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)
    neurons_1st_hidden = round(neurons_1st_hidden)
    neurons_other_hidden_1 = round(neurons_other_hidden_1)
    neurons_other_hidden_2 = round(neurons_other_hidden_2)
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    def nn_cl_fun():
        nn = Sequential()
        nn.add(Dense(neurons_1st_hidden, input_dim=10, activation='relu'))
        for i in range(layers1):
            nn.add(Dense(neurons_other_hidden_1, activation='relu'))
        for i in range(layers2):
            nn.add(Dense(neurons_other_hidden_2, activation='relu'))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=10)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(nn, train_feature_scaled, train_label, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [35]:
# The following code searches for the optimum hyperparameters and layers for the Neural Network model
params_nn2 ={
    'neurons_1st_hidden': (10, 500),
    'neurons_other_hidden_1':(10,500),
    'neurons_other_hidden_2':(10,500),
    'learning_rate':(0.00001, 0.001),
    'batch_size':(10, 100),
    'epochs':(10, 500),
    'layers1':(1,5),
    'layers2':(1,5),
}
# Run Bayesian Optimization
# 'normalization':(0,1), # 이건 사용 안함

nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=10, n_iter=10)

|   iter    |  target   | batch_... |  epochs   |  layers1  |  layers2  | learni... | neuron... | neuron... | neuron... |
-------------------------------------------------------------------------------------------------------------------------
2/2 [==============================] - 0s 0s/step
|  1        |  0.8669   |  65.1     |  92.84    |  2.744    |  4.077    |  0.000302 |  83.09    |  21.01    |  215.9    |
2/2 [==============================] - 0s 2ms/step
|  2        |  0.8511   |  31.48    |  175.5    |  4.963    |  1.951    |  9.038e-0 |  338.1    |  314.4    |  144.4    |
2/2 [==============================] - 0s 3ms/step
|  3        |  0.8883   |  51.96    |  68.0     |  1.296    |  4.603    |  0.000796 |  421.9    |  409.5    |  495.6    |
2/2 [==============================] - 0s 2ms/step
|  4        |  0.872    |  61.95    |  408.7    |  2.685    |  1.11     |  0.000459 |  61.61    |  410.4    |  351.9    |
2/2 [==============================] - 0s 3ms/step
|  5        | 

In [36]:
# Fitting Neural Network
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons_1st_hidden'] = round(params_nn_['neurons_1st_hidden'])
params_nn_['neurons_other_hidden_1'] = round(params_nn_['neurons_other_hidden_1'])
params_nn_['neurons_other_hidden_2'] = round(params_nn_['neurons_other_hidden_2'])
params_nn_

{'batch_size': 75,
 'epochs': 193,
 'layers1': 3,
 'layers2': 1,
 'learning_rate': 0.0006138233455413297,
 'neurons_1st_hidden': 371,
 'neurons_other_hidden_1': 143,
 'neurons_other_hidden_2': 128}

In [37]:
for i, res in enumerate(nn_bo.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': 0.8668563300142248, 'params': {'batch_size': 65.09531580558567, 'epochs': 92.84417962936185, 'layers1': 2.744236077484681, 'layers2': 4.077049890092491, 'learning_rate': 0.00030237205135884743, 'neurons_1st_hidden': 83.08984899834999, 'neurons_other_hidden_1': 21.014379026639265, 'neurons_other_hidden_2': 215.9100012096333}}
Iteration 1: 
	{'target': 0.8510668563300143, 'params': {'batch_size': 31.481392712180146, 'epochs': 175.45153402550827, 'layers1': 4.962849858523515, 'layers2': 1.9509058156000911, 'learning_rate': 9.038073285669945e-05, 'neurons_1st_hidden': 338.1041167408486, 'neurons_other_hidden_1': 314.40903050634154, 'neurons_other_hidden_2': 144.38422980119404}}
Iteration 2: 
	{'target': 0.8883357041251779, 'params': {'batch_size': 51.95992689054758, 'epochs': 68.00019805040895, 'layers1': 1.2958302565728168, 'layers2': 4.603096716642735, 'learning_rate': 0.0007960229348748321, 'neurons_1st_hidden': 421.8791278748978, 'neurons_other_hidden_1': 409.

In [38]:
def nn_cl_fun_2():
    nn = Sequential()
    nn.add(Dense(params_nn_['neurons_1st_hidden'], input_dim=10, activation='relu'))
    for i in range(params_nn_['layers1']):
        nn.add(Dense(params_nn_['neurons_other_hidden_1'], activation='relu'))
    # if params_nn_['dropout'] > 0.5:
    #     nn.add(Dropout(params_nn_['dropout_rate'], seed=123))
    for i in range(params_nn_['layers2']):
        nn.add(Dense(params_nn_['neurons_other_hidden_2'], activation='relu'))
    nn.add(Dense(1, activation='sigmoid'))
    
    optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)
    
    nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return nn

es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=10)
nn = KerasClassifier(build_fn=nn_cl_fun_2, epochs=params_nn_['epochs'], batch_size=params_nn_['batch_size'],
                         verbose=0)


# train_set : validation_set : test_set = 80 : 10 : 10 
train_feature_scaled, validation_feature_scaled, train_label, validation_label = train_test_split(train_feature_scaled, train_label, test_size = 0.10,random_state=0)

nn.fit(train_feature_scaled, train_label, validation_data=(validation_feature_scaled, validation_label), verbose=1)

Epoch 1/193
3/3 [==============================] - 1s 62ms/step - loss: 0.6845 - accuracy: 0.5266 - val_loss: 0.6837 - val_accuracy: 0.5789
Epoch 2/193
3/3 [==============================] - 0s 13ms/step - loss: 0.6543 - accuracy: 0.6686 - val_loss: 0.6638 - val_accuracy: 0.7895
Epoch 3/193
3/3 [==============================] - 0s 13ms/step - loss: 0.6169 - accuracy: 0.7456 - val_loss: 0.6334 - val_accuracy: 0.7895
Epoch 4/193
3/3 [==============================] - 0s 13ms/step - loss: 0.5669 - accuracy: 0.8166 - val_loss: 0.5800 - val_accuracy: 0.7895
Epoch 5/193
3/3 [==============================] - 0s 13ms/step - loss: 0.4956 - accuracy: 0.8343 - val_loss: 0.5122 - val_accuracy: 0.8421
Epoch 6/193
3/3 [==============================] - 0s 14ms/step - loss: 0.4208 - accuracy: 0.8817 - val_loss: 0.4350 - val_accuracy: 0.8421
Epoch 7/193
3/3 [==============================] - 0s 13ms/step - loss: 0.3365 - accuracy: 0.9053 - val_loss: 0.3736 - val_accuracy: 0.7895
Epoch 8/193
3/3 [===

In [39]:
nn.score(test_feature_scaled, test_label)

0.7916666865348816